In [26]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'distutils'

In [27]:
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bboxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        bboxes.append([xmin, ymin, xmax, ymax])
    return bboxes

In [28]:
def preprocess_image(image_path, target_size=(64, 64)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    image = cv2.resize(image, target_size)  # Resize image
    image = image / 255.0  # Normalize pixel values
    return image

def load_dataset(images_dir, annotations_dir):
    images = []
    labels = []
    
    image_files = os.listdir(images_dir)
    
    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        annotation_file = os.path.splitext(image_file)[0] + ".xml"
        annotation_path = os.path.join(annotations_dir, annotation_file)

        # Preprocess the image
        image = preprocess_image(image_path)

        # Get the bounding boxes
        if os.path.exists(annotation_path):
            bboxes = parse_annotation(annotation_path)
            
            image_height, image_width = image.shape[:2]
            
            for bbox in bboxes:
                xmin, ymin, xmax, ymax = bbox

                xmin = max(0, xmin)
                ymin = max(0, ymin)
                xmax = min(image_width, xmax)
                ymax = min(image_height, ymax)

                if xmax > xmin and ymax > ymin:
                    plate_image = image[ymin:ymax, xmin:xmax]  # Crop the plate region
                    
                    if plate_image.size == 0:
                        print(f"Warning: Cropped image is empty for {image_file}, bbox: {bbox}")
                        continue

                    try:
                        plate_image_resized = cv2.resize(plate_image, (64, 64))  # Resize to (64, 64)
                        images.append(plate_image_resized)
                        labels.append(1)  # Label 1 for plate detected
                    except Exception as e:
                        print(f"Error resizing image {image_file} with bbox {bbox}: {str(e)}")
                        continue
        else:
            continue

    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

# Load data
images_dir = 'archive/images'  # Path to your images folder
annotations_dir = 'archive/annotations'  # Path to your annotations folder
images, labels = load_dataset(images_dir, annotations_dir)
images = np.expand_dims(images, axis=-1)  # Add channel dimension for grayscale

In [29]:
def create_cnn_model(input_shape):
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the layers
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification (plate/no plate)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [15]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [16]:
# Build the CNN model
model = create_cnn_model((64, 64, 1))

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Save the trained model
model.save('vehicle_plate_recognition_cnn.h5')

NameError: name 'Sequential' is not defined

In [30]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
def predict_plate(image_path, model):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    prediction = model.predict(image)
    return np.round(prediction)

# Example prediction
test_image = 'test_image.jpg'  # Path to a test image
result = predict_plate(test_image, model)
print(f'Plate detected: {result}')